In [ ]:
# Start writing code here...from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
spotify_songs = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')


Q1 = np.percentile(spotify_songs.duration_ms, 25,
                   interpolation = 'midpoint')
Q3 = np.percentile(spotify_songs.duration_ms, 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1

"""
mask = (spotify_songs.duration_ms < (Q3+1.5*IQR))
mask2 = spotify_songs.duration_ms > (Q1-1.5*IQR)
spotify_data_cleaned = spotify_songs.loc[mask]
spotify_data_cleaned = spotify_data_cleaned.loc[mask2]
"""
spotify_data_cleaned = spotify_songs
spotify_data_cleaned.track_album_release_date.str[:4]
spotify_data_cleaned.track_album_release_date.str[5:7]
spotify_data_cleaned = spotify_data_cleaned.assign(
    track_album_release_month = spotify_data_cleaned.track_album_release_date.str[5:7],
    track_album_release_year = spotify_data_cleaned.track_album_release_date.str[:4],
)
spotify_data_cleaned.drop(columns="track_album_release_date",inplace=True)
spotify_data_cleaned.track_album_release_year = spotify_data_cleaned.track_album_release_year.astype("int64")
"""
mask = (spotify_data_cleaned.track_popularity > 0)
spotify_data_cleaned = spotify_data_cleaned.loc[mask]
"""



'\nmask = (spotify_data_cleaned.track_popularity > 0)\nspotify_data_cleaned = spotify_data_cleaned.loc[mask]\n'

In [ ]:
spotify_data_cleaned

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_album_release_month,track_album_release_year
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,194754,06,2019
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.0373,0.072400,0.004210,0.3570,0.6930,99.972,162600,12,2019
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,0,0.0742,0.079400,0.000023,0.1100,0.6130,124.008,176616,07,2019
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,169093,07,2019
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,189052,03,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,7bxnKAamR3snQ1VGLuVfC1,City Of Lights - Official Radio Edit,Lush & Simon,42,2azRoBBWEEEYhqV6sb7JrT,City Of Lights (Vocal Mix),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375,04,2014
32829,5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120,03,2013
32830,7ImMqPP3Q1yfUHvsdn7wEo,Sweet Surrender - Radio Edit,Starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,Sweet Surrender (Radio Edit),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112,04,2014
32831,2m69mhnfQ1Oq6lGtXuYhgX,Only For You - Maor Levi Remix,Mat Zo,15,1fGrOkHnHJcStl14zNx8Jy,Only For You (Remixes),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432,01,2014


In [ ]:
artists_mask = (spotify_data_cleaned.track_artist == "ABBA")
data_billie_eilish = spotify_data_cleaned.loc[artists_mask]
data_billie_eilish


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_album_release_month,track_album_release_year
21222,4NtUY5IGzHCaqfZemmAu56,Dancing Queen,ABBA,72,1M4anG49aEs4YimBdj96Oy,Arrival,School Dance 2019 (Squeaky Clean),0SqaMfNsngZCpPw0UuelA7,latin,latin hip hop,...,1,0.0403,0.382,0.00166,0.76,0.752,100.812,230693,,1976


In [ ]:
track_artist_value = spotify_data_cleaned['track_artist'].value_counts()
print(track_artist_value)
track_artist_value[:600]

Martin Garrix       161
Queen               136
The Chainsmokers    123
David Guetta        110
Don Omar            102
                   ... 
Circa Waves           1
Saavan                1
Neneh Cherry          1
Jac & Harri           1
Jasper Loco           1
Name: track_artist, Length: 10692, dtype: int64


Martin Garrix             161
Queen                     136
The Chainsmokers          123
David Guetta              110
Don Omar                  102
                         ... 
Michael Learns To Rock     10
Kendra Morris              10
DIVINE                     10
Cosculluela                10
Shai                       10
Name: track_artist, Length: 600, dtype: int64

In [ ]:
import seaborn as sns
sns.histplot(x=spotify_data_cleaned.t)

AttributeError: 'DataFrame' object has no attribute 't'

- mq (ou pas) si on connait l'artiste on peut deviner le genre de sa musique (peut être à partir d'un certain nombres de musique qu'il a produites)
- cependant il y a pas l'air d'avoir bcp d'artistes qui ont fait bcp de sons (deja il y a 10 000 artistes differents et en plus il y en a que 600 qui ont plus de 10 titres, bon du coup est ce qu'il y a bcp de chansons d'artiste inconnu jsp faudra que j'analyse)

peut être faire un test pour voir si les gros groupes font de meilleurs chansons / plutôt que les groupes/chanteurs n'ont qu'une chanson (est ce qu'on peut expliquer une dependance ? Test du chi2 coucou sy02)


In [ ]:
spotify_data_cleaned["playlist_name"].value_counts()


Indie Poptimism                                              308
2020 Hits & 2019  Hits – Top Global Tracks 🔥🔥🔥               247
Permanent Wave                                               244
Hard Rock Workout                                            219
Ultimate Indie Presents... Best Indie Tracks of the 2010s    198
                                                            ... 
Big White Room-Jessie-J                                        7
CSR 103:9 (GTA: SA)                                            7
TOP 50 GLOBAL 2020 UPDATED WEEKLY 🌍🎶 WORLDWIDE                 6
ALPAS Music Festival                                           3
Post-Teen Pop                                                  1
Name: playlist_name, Length: 449, dtype: int64

peut être qu'il y a des choses a faire aussi côté playlist : attention aux niveau prédiction !!!

In [ ]:
test = spotify_data_cleaned["track_id"].value_counts()
test[test == 1].shape[0]/spotify_data_cleaned.shape[0]

0.7672159108214297

pas que j'oublie : 
est ce que c'est normal d'aovir meme score knn lda qda ...
acp 6 composantes mais 20 % dans chq 
year vraiment quali ? en vrai vraiment utile en temps que quali

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c2ad4461-7f35-42f5-b2d9-f686e8952f67' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>